<a href="https://colab.research.google.com/github/fener95/GEE-bugisu-project/blob/main/Fig1_BugisuProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate(auth_mode= 'notebook',  #force =True,
                #scopes='https://www.googleapis.com/auth/earthengine'
                )

ee.Initialize(project='takehome-exam-bugisu-project')

In [ ]:
# if requested by gee, click on the secret-key (left panel); name: EE-PROJECT-ID; value: takehome-exam-bugisu-project

# then run:
"""
from google.colab import userdata
userdata.get('EE_PROJECT_ID')
"""

In [ ]:
import geemap
Map = geemap.Map(center=[1.1, 34.2], zoom=9)

In [ ]:
#other libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import datetime
from shapely.geometry import mapping
#-----CARTOEE-----
!pip install cartopy scipy
import cartopy
from geemap import cartoee
import cartopy.crs as ccrs
from matplotlib.lines import Line2D

# Set the aoi from my asset (should work, in case does not: use the GAUL dataset and filter for ADM1_NAME: sironko, bududa, manafwa, kapchorwa, mbale, bulambuli, kween, bukwo)

In [ ]:
aoi = \
ee.FeatureCollection("projects/takehome-exam-bugisu-project/assets/bugisu_subRegion")
geometry = aoi.geometry()
#Map.addLayer(aoi.style(**{'color': 'red', 'fillColor': '00000000'}), {}, 'Bugisu sub-region')

# Elevation and slopes

In [ ]:
DEM = ee.Image("CGIAR/SRTM90_V4")  # this is the elevation dataset at 90m spt res
# Clip the DEM image to the AOI
clipped_dem = DEM.clip(geometry)  # clip to the extent of the area

# Define the terrain object
terrain = ee.Algorithms.Terrain(clipped_dem)
elevation = terrain.select('elevation')
# palette and vis params for elevation
ele_vis = {
    'min': 0,
    'max': 4323,
    'palette': [ '0000ff', '00ff00', 'ffff00', 'ff0000', 'ffffff']
}

# Define the slope image
slope = terrain.select('slope')

# slope vis params
slopeVis = {
    'min': 0,
    'max': 90,
    'palette': ['blue', 'green', 'yellow', 'red']
}
#Map.addLayer(slope, slopeVis, 'Slope', False)
#Map.addLayer(elevation, ele_vis, 'elevation (m)',0)

# (OPTIONAL) Export images to drive for QGIS or arcgis

In [ ]:
image = elevation # or slope

# Set up Modify here and insert the desired input
description='bugisu-elev-90m'
folder = 'rasters'
scale=90
crs='EPSG:4326'

# run task
task = ee.batch.Export.image.toDrive(
    image=image,
    description=description,
    folder=folder,
    region=geometry,
    scale=scale,
    crs=crs
)
#task.start()

# Create threshold of elevation zones

In [ ]:
scale = elevation.projection().nominalScale().getInfo()  # set the scale from the scale of elevation
# Remap values.
elevZones = ee.Image(1) \
    .where(elevation.gt(800).And(elevation.lte(1000)), 2) \
    .where(elevation.gt(1000).And(elevation.lte(1100)),3) \
    .where(elevation.gt(1100).And(elevation.lte(1200)), 4) \
    .where(elevation.gt(1200).And(elevation.lte(1400)), 5) \
    .where(elevation.gt(1400).And(elevation.lte(1600)), 6) \
    .where(elevation.gt(1600).And(elevation.lte(1800)), 7) \
    .where(elevation.gt(1800).And(elevation.lte(2000)), 8) \
    .where(elevation.gt(2000).And(elevation.lte(2200)), 9) \
    .where(elevation.gt(2200).And(elevation.lte(2400)), 10) \
    .where(elevation.gt(2400).And(elevation.lte(2600)), 11) \
    .where(elevation.gt(2600).And(elevation.lte(2800)), 12) \
    .where(elevation.gt(2800).And(elevation.lte(3000)), 13)\
    .where(elevation.gt(3000).And(elevation.lte(4300)), 14).clip(geometry).reproject(crs='EPSG:4326', scale=scale)

# Define the elevation zones color palette
paletteElev = [
    '006400', # DarkGreen
    '228B22', # ForestGreen
    'ADFF2F', # GreenYellow
    'FFFF00', # Yellow
    'FFD700', # Gold
    'FFA500', # Orange
    'FF8C00', # DarkOrange
    'FF4500', # OrangeRed
    'FF0000', # Red
    '8B0000',  # DarkRed
    '800080', # Purple
    'FF00FF', # Magenta
    'FFC0CB', # Pink
    'cyan'  # cyan
]
#Map.addLayer(elevZones, {'min': 1, 'max': 14, 'palette': paletteElev}, 'Elevation Zones',0)

######OPTIONAL
"""
# Convert the classified image to vector data
zones = elevZones.reduceToVectors(
    reducer=ee.Reducer.mode(),  # Mode for categorical data
    scale=scale,  # Adjust according to your DEM resolution
    geometryType='polygon',  # Output as polygons
    eightConnected=False,  # Optional, depends on your needs
)
"""

# Mapping with cartoee

In [ ]:
# define the legend dict

legend_dict = {
    '800-1000 (m)': '006400',
    '1000-1100 (m)': '228B22',
    '1100-1200 (m)': 'ADFF2F',
    '1200-1400 (m)': 'FFFF00',
    '1400-1600 (m)': 'FFD700',
    '1600-1800 (m)': 'FFA500',
    '1800-2000 (m)': 'FF8C00',
    '2000-2200 (m)': 'FF4500',
    '2200-2400 (m)': 'FF0000',
    '2400-2600 (m)': '8B0000',
    '2600-2800 (m)': '800080',
    '2800-3000 (m)': 'FF00FF',
    '3000-3400 (m)': 'FFC0CB',
    '3400-4300 (m)': '00FFFF'
}
# Create the legend items
legend = []
for elevation, color in legend_dict.items():
    item = Line2D(
        [],
        [],
        marker="o",
        color='#' + color,
        label=elevation,
        markerfacecolor='#' + color,
        markersize=5,
        linestyle='',
    )
    legend.append(item)

# Create the figure and axis
fig, ax = plt.subplots(figsize=(7, 5))

In [ ]:
# Draw one rectangle that inscribe the AOI on the map using the Drawing tools before executing this code block
area = Map.user_roi
#alternatively use these:
# Example polygon coordinates
polygon_coords = [
    [34.054871, 0.722332],
    [34.054871, 1.658704],
    [34.859619, 1.658704],
    [34.859619, 0.722332],
    [34.054871, 0.722332]
]

# Create a polygon
polygon = ee.Geometry.Polygon(polygon_coords)

# Compute bounding box
bounds = polygon.bounds()

# Extract coordinates from bounding box
coords = bounds.getInfo()['coordinates'][0]
minLon, minLat = coords[0]
maxLon, maxLat = coords[2]

# Define the rectangle
area = [minLon, minLat, maxLon, maxLat]

In [ ]:
vis_params = {
    'min': 1,
    'max': 14,
    'palette': paletteElev}

In [ ]:
# Create a map using cartoee
fig = plt.figure(figsize=(7, 5))
ax = cartoee.get_map(elevZones, region=area, vis_params=vis_params)
# add grid lines to the map at a specified interval
cartoee.add_gridlines(ax, interval=0.1, xtick_rotation=0, linestyle=":")
# title
ax.set_title(label='Elevation Zones Bugisu', fontsize=15)
# add scale bar
scale_bar_dict = {
    "length": 10,
    "xy": (0.01, 0.95),
    "linewidth": 2,
    "fontsize": 10,
    "color": "black",
    "unit": "km",
    "ha": "center",
    "va": "bottom",
}
cartoee.add_scale_bar_lite(ax, **scale_bar_dict)
# Add the legend to the map.
cartoee.add_legend(
    ax,
    legend_elements=legend,
    font_size=6,
    title='Elevation Zones',
    title_fontsize=8,
    loc='lower right',
)

plt.show()

In [ ]:
# download to your local env
cartoee.savefig(fig, 'elevZones_Elgon.jpg', dpi=300)